In [3]:
%reload_ext autoreload
%autoreload 2

import os
import argparse
import sys
from time import time

sys.path.append(os.path.join(os.environ['GORDON_REPO_DIR'], 'utilities'))
from utilities2015 import *

from matplotlib.path import Path
%matplotlib inline

import scipy
from sklearn import svm
from sklearn.metrics import confusion_matrix
from multiprocessing import Pool
import random
import pickle
from sklearn.cluster import KMeans

In [4]:
M = 200;
L = 2;
classes = ['5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL' , 'BackG'];
testDir = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_patches/MD594_byROI/';

In [5]:
vocabulary = np.load('/oasis/projects/nsf/csd395/ruogu/svm3/vocabulary.npy');
clf = pickle.load(open('/oasis/projects/nsf/csd395/ruogu/svm3/clf.p', "rb"));
training_X = np.load('/oasis/projects/nsf/csd395/ruogu/svm3/training_X.npy');

In [14]:
kmeans = KMeans(init='random', n_clusters=M, n_init=10);
kmeans.fit(vocabulary);

In [17]:
# Input: an rgb image filen name, a kmeans object, M and L
def getHistogram(fileName, patchSize=16, stride=56):
    # Change image to gray scale
    image = imread(testDir + fileName);
    img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY);
    # Generate key points
    height = img.shape[0];
    width = img.shape[1];
    xv, yv = np.meshgrid(np.arange(patchSize / 2, width - patchSize / 2, stride),
                         np.arange(patchSize / 2, height - patchSize / 2, stride),
                         indexing='ij');
    sample_points = np.c_[xv.flat, yv.flat];
    cv_keypoints = [cv2.KeyPoint(x,y, patchSize) for x, y in sample_points];
    # Get SIFT descriptor.
    sift = cv2.SIFT();
    _, descriptors = sift.compute(img, cv_keypoints);
    # Map key points with type.
    cluster_labels = kmeans.predict(descriptors);
    cluster_labels = cluster_labels;
    # Level 0
    weight_0 = 1.0 / (2**L);
    histogram = np.bincount(cluster_labels, minlength = M);
    histogram = histogram * weight_0;
    # Other levels
    for l in range(1, L + 1):
        weight = 1.0 / (2**(L - l + 1));
        grid_size_x = width / (2**l);
        grid_size_y = height / (2**l);
        grid_boundaries_x = range(0, width, grid_size_x);
        grid_boundaries_y = range(0, height, grid_size_y);
        number_of_grid = (2**l)**2;
    
        grid_crs = sample_points / [grid_size_x, grid_size_y];
        grid_cols = grid_crs[:,0];  # x
        grid_rows = grid_crs[:,1];  # y
    
        keypoint_grid_indices = grid_rows * (2**l) + grid_cols;
        hists = np.zeros(shape=(number_of_grid, M), dtype=np.int);
        for ki, label in zip(keypoint_grid_indices, cluster_labels):
            hists[ki][label] += 1;
        hists = hists * weight;
        histogram = np.hstack((histogram, hists.flatten()));
    return ((histogram / len(sample_points)), fileName); # Normalization

In [18]:
section_id = range(157, 167);
testDir = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_patches/MD594_byROI/' + str(section_id[0]).zfill(4)+ '/roi1/';
saveDir = '/oasis/projects/nsf/csd395/ruogu/svm3/test/';
# Get histograms
fileNames = os.listdir(testDir);
print len(fileNames);

16938


In [20]:
t = time();
pool = Pool(processes=8);
result = pool.map(getHistogram, fileNames);
print "Generating histogram takes {} sec.".format(time() - t);

Generating histogram takes 80.6911289692 sec.


In [28]:
testing_X = [x[0] for x in result];
names = [x[1] for x in result];

In [31]:
testing_X = np.asarray(test_X);
n_testing = test_X.shape[0];
n_training = training_X.shape[0];

In [25]:
supp_indices = clf.support_;
supp_vecs = training_X[clf.support_];

In [34]:
# Test in parallel.
def test_job(i):
    xi = testing_X[i];
    vals = 0.5 * np.sum(supp_vecs + xi - np.abs(supp_vecs - xi), axis=1); # Equals to min(xi, xj)
    his = np.zeros(n_training);
    his[supp_indices] = vals;
    return (i, his);

start = time();
pool = Pool(processes=16);
result = pool.map(test_job, range(0, n_testing));
end = time();
print "Kernel calculation takes {} sec.".format(end - start);

Kernel calculation takes 1469.59512997 sec.


In [35]:
sorted(result, key=lambda x: x[0]);
testKernel = np.asarray([v[1] for v in result]);
testKernel = testKernel.reshape(n_testing, n_training);
predict_labels_test = clf.predict(testKernel);

In [36]:
print len(predict_labels_test);

16938


In [51]:
output_file = open('/oasis/projects/nsf/csd395/ruogu/svm3/test/' + str(section_id[0]).zfill(4) + '.txt', 'w');

In [52]:
for i in range(0, n_testing):
    line = names[i] + " " + str(predict_labels_test[i]) + '\n';
    output_file.write(line);

In [53]:
output_file.close();